In [1]:
SEED=185

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix

In [3]:
DATA='data/mamografias.csv'

# Función para leer los datos
def readData(data_file):
    return pd.read_csv(data_file,sep=',', na_values='?')

data = readData(DATA) # Lectura de los datos

data.rename(columns = {'BI-RADS':'BiRads'}, inplace = True) # Para poder referirnos a esta columna como data.BiRads
data.BiRads.replace(0,pd.NA,inplace=True) # BiRads 0 significa radiografía insuficiente
data.Shape.replace('N',pd.NA,inplace=True) # Lo mismo pasa con Shape N
data.replace(pd.NA, np.nan, inplace=True)

data=data.dropna() # Eliminamos las instancias con valores perdidos
data.shape[0] # Nos quedamos con 825 instancias

825

In [4]:
data.describe() # Baja desviación en BiRads y en Density

,BiRads,Age,Margin,Density
count,825.000000,825.000000,825.000000,825.000000
mean,4.360000,55.738182,2.806061,2.915152
std,0.601455,14.683532,1.568412,0.351937
min,2.000000,18.000000,1.000000,1.000000
25%,4.000000,46.000000,1.000000,3.000000
50%,4.000000,57.000000,3.000000,3.000000
75%,5.000000,66.000000,4.000000,3.000000
max,6.000000,96.000000,5.000000,4.000000


In [5]:
# Vemos que pasa, también con Shape, que al no ser numérica no se mostraba
from collections import Counter
print(Counter(data.BiRads)) # La mayoría de instancias tienen 4 o 5
print(Counter(data.Density)) # La mayoría de instancias tienen 3
print(Counter(data.Shape)) # Su distribuición está bastante lejos de ser degenerada

Counter({4.0: 468, 5.0: 317, 3.0: 24, 6.0: 9, 2.0: 7})
Counter({3.0: 750, 2.0: 56, 1.0: 11, 4.0: 8})
Counter({'I': 375, 'R': 190, 'O': 179, 'L': 81})


In [6]:
# La distribución de la característica densidad es prácticamente degenerada en 3
data.drop('Density', axis='columns',inplace=True)
data.BiRads=data.BiRads.replace(6,5).replace(2,4).replace(3,4)
print(Counter(data.BiRads))
data.head()

Counter({4.0: 499, 5.0: 326})


,BiRads,Age,Shape,Margin,Severity
0,5.0,67.0,L,5.0,maligno
2,5.0,58.0,I,5.0,maligno
3,4.0,28.0,R,1.0,benigno
8,5.0,57.0,R,5.0,maligno
10,5.0,76.0,R,4.0,maligno


In [7]:
# Sklearn necesita datos numéricos (aunque sean nominales)
le = preprocessing.LabelEncoder()
data.Shape = le.fit_transform(data.Shape)
data.Severity = le.fit_transform(data.Severity)
print(le.inverse_transform([0,1])) # Consideraremos maligno como la clase positiva

['benigno' 'maligno']


In [8]:
data.corr() # Severity tiene una correlación similar con todas, Shape y Margin están bastante correladas

,BiRads,Age,Shape,Margin,Severity
BiRads,1.000000,0.400823,-0.552308,0.574538,0.669436
Age,0.400823,1.000000,-0.377555,0.420682,0.453140
Shape,-0.552308,-0.377555,1.000000,-0.738531,-0.564691
Margin,0.574538,0.420682,-0.738531,1.000000,0.576492
Severity,0.669436,0.453140,-0.564691,0.576492,1.000000


In [9]:
dataArray=np.array(data)
dataArray

array([[ 5., 67.,  1.,  5.,  1.],
       [ 5., 58.,  0.,  5.,  1.],
       [ 4., 28.,  3.,  1.,  0.],
       ...,
       [ 4., 64.,  0.,  5.,  0.],
       [ 5., 66.,  0.,  5.,  1.],
       [ 4., 62.,  1.,  3.,  0.]])

In [10]:
# Separamos en datos y target (label)
x=dataArray[:,:-1]
y=dataArray[:,-1]

In [11]:
# Para calcular la matriz de confusión usando validación cruzada sumamos las matrices obtenidas en las distintas particiones
# https://stats.stackexchange.com/questions/147175/how-is-the-confusion-matrix-reported-from-k-fold-cross-validation
# https://stackoverflow.com/questions/40057049/using-confusion-matrix-as-scoring-metric-in-cross-validation-in-scikit-learn
def KFoldConfusionMatrix(model, data, target):
    conf_matrix_list_of_arrays = []
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    for train_index, test_index in kf.split(data):
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = target[train_index], target[test_index]
        model.fit(X_train, y_train)
        conf_matrix = confusion_matrix(y_test, model.predict(X_test))
        conf_matrix_list_of_arrays.append(conf_matrix)
    return np.sum(conf_matrix_list_of_arrays, axis=0)

In [12]:
""" La matriz de confusión aparece como
          Pred:    0  1
(Benigno) Real=0: TN FP
(Maligno) Real=1: FN TP
"""
# Quiero ponerla: TP, TN, FP, FN
def mcToArray(mc):
    return ', '.join(list(map(str,[mc[1][1], mc[0][0], mc[0][1], mc[1][0]])))

In [13]:
# Dummy: siempre maligno (aunque benigno sea más frecuente)
dummy=DummyClassifier(strategy='constant',constant=1)
dummy_cf=KFoldConfusionMatrix(dummy, x, y)
print(dummy_cf)

[[  0 425]
 [  0 400]]


In [14]:
# Decision Tree
dt=tree.DecisionTreeClassifier(random_state=SEED)
dt_cf=KFoldConfusionMatrix(dt, x, y)
print(dt_cf)

[[349  76]
 [107 293]]


In [15]:
# Gaussian Naive Bayes
gnb=GaussianNB()
gnb_cf=KFoldConfusionMatrix(gnb, x, y)
print(gnb_cf)

[[347  78]
 [ 54 346]]


In [16]:
# Multinomial Naive Bayes
mnb=MultinomialNB()
mnb_cf=KFoldConfusionMatrix(mnb, x, y)
print(mnb_cf)

[[324 101]
 [ 71 329]]


In [17]:
# Random Forest
rf=RandomForestClassifier(n_jobs=4, random_state=SEED)
rf_cf=KFoldConfusionMatrix(rf, x, y)
print(rf_cf)

[[345  80]
 [ 87 313]]


In [18]:
# KNN
knn=KNeighborsClassifier() # K=5 por defecto
knn_cf=KFoldConfusionMatrix(knn, x, y)
print(knn_cf)

[[335  90]
 [ 72 328]]


In [19]:
# Neural Network
rn=MLPClassifier(max_iter=500,random_state=SEED) # Max_iter=500 porque recibí warning de convergencia
rn_cf=KFoldConfusionMatrix(rn, x, y)
print(rn_cf)

[[338  87]
 [ 77 323]]


In [20]:
with open("results/results_features.csv",'w+') as outfile:
    outfile.write('\n'.join([
            mcToArray(dummy_cf),
            mcToArray(dt_cf),
            mcToArray(gnb_cf),
            mcToArray(mnb_cf),
            mcToArray(rf_cf),
            mcToArray(knn_cf),
            mcToArray(rn_cf)
        ]))